In [ ]:
import numpy as np 
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import pickle as cPickle
from os.path import exists
from pathlib import Path
import csv
import datetime
import pytz
import plotly.express as px
import os

from pathlib import Path 
import time

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/testing_data_all_20220523.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Columns: 299 entries, Unnamed: 0 to wb2sum
dtypes: float64(290), int64(4), object(5)
memory usage: 1.1+ MB


In [ ]:
usgs_reg = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/USGS_region_lookup.csv')
usgs_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NA_L3CODE   11 non-null     object
 1   USGS_Group  11 non-null     object
dtypes: object(2)
memory usage: 304.0+ bytes


In [6]:
region_dict = dict(zip(usgs_reg.NA_L3CODE, usgs_reg.USGS_Group))
test['NewEco3'] = test['NewEco3'].replace(region_dict)
print(test['NewEco3'].unique())

['Interior Mountains']


In [7]:
test = test.loc[(test['NewEco3'] == 'Interior Mountains')] 
test

,Unnamed: 0,year,month,flow_cfs,comid,statistic,variable,ECO3,NHDPLUSREG,DRAIN_SQKM,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
8,8,2021,11,163.620000,8922715,mean,observed,5.0,16.0,715.7367,...,28.349367,33.610842,18.383408,8.003475,287.289841,213.320097,207.804507,177.042778,76.729239,64.767735
9,9,2021,11,0.099667,8922761,mean,observed,5.0,16.0,14.4684,...,18.181329,7.957989,0.895567,2.039243,216.625117,155.729993,149.344575,29.338021,9.455206,7.933217
10,10,2021,11,39.130000,8924041,mean,observed,5.0,16.0,169.9299,...,36.468143,45.949072,36.986141,11.781768,300.406668,230.309273,228.717690,178.004598,83.494384,76.676339
11,11,2021,11,14.667667,8941733,mean,observed,5.0,16.0,19.8864,...,66.703961,68.531156,46.261531,19.242495,340.350991,317.371088,315.856014,175.849878,121.171483,117.272201
12,12,2021,11,26.830000,8941693,mean,observed,5.0,16.0,30.4992,...,79.209620,82.813188,57.258220,20.317768,371.908117,345.868207,345.121565,224.182809,125.632375,118.648726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,41,2022,4,7.990000,14930711,mean,observed,5.0,18.0,58.9752,...,2.038410,3.680669,7.349272,14.938212,401.566098,62.735574,62.735574,144.328568,72.023125,34.058706
455,42,2022,4,15.610000,14930789,mean,observed,5.0,18.0,251.3952,...,0.832659,1.516839,3.030831,6.288357,341.165533,45.179699,45.179699,92.643116,38.165263,15.586369
457,44,2022,4,1060.400000,19791955,mean,observed,5.0,18.0,4348.9110,...,23.558600,27.909581,24.916684,24.536143,550.217586,24.191563,23.938548,219.205578,83.658204,32.312229
461,48,2022,4,0.717667,17078425,mean,observed,5.0,18.0,41.6691,...,2.298312,4.569962,9.211781,18.984303,517.934985,48.567597,45.926691,235.676675,100.669842,44.399818


In [8]:
#right now we have Nov 2021 - Apr 2022 data
test['month'].unique()

array([11, 12,  1,  2,  3,  4])

In [9]:
#no null columns
test = test.dropna(axis=0,how='any')
test.isnull().sum().sum()

0

In [10]:
test['flow_cfs'] = test['flow_cfs'] / test['DRAIN_SQKM']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
#dropping inessential columns
col = ['comid','Unnamed: 0','year','statistic','variable','NHDPLUSREG','comidyear','ECO3','AggEcoRegion','NewEco3'] 
df = test.drop(columns = col)

#no need label encode as only have one region xeric
#label encoding
# cols_encoding = ['NewEco3']
# labelencoder = LabelEncoder()
# df['NewEco3_label'] = labelencoder.fit_transform(df['NewEco3'])
# df = df.drop(columns=cols_encoding)
# df.reset_index(drop=True, inplace=True)

In [12]:
def x_test(month):  
  test1 = df[df['month']==month]
  x_test = test1.drop(columns = ['flow_cfs']).copy()
  return x_test

def y_test(month):
  test1 = df[df['month']==month]
  y_test = test1.flow_cfs.values
  return y_test

November 2021

In [13]:
pickled_model_usgs_11 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Interior_M-all-comids)/USGS_model_reduced_11_r2_0.8355_2022-09-19 13:03:14.970221-07:00.pkl', 'rb'))
print(pickled_model_usgs_11)

RandomForestRegressor(max_depth=5, max_features=16, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [14]:
rf_var = pickled_model_usgs_11.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_11
predictions = rf.predict(x_test(11)) 
r2_test = r2_score(y_test(11)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.05


December 2021

In [15]:
pickled_model_usgs_12 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Interior_M-all-comids)/USGS_model_reduced_12_r2_0.8883_2022-09-19 13:04:59.754792-07:00.pkl', 'rb'))
print(pickled_model_usgs_12)

RandomForestRegressor(max_depth=5, max_features=16, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [16]:
rf_var = pickled_model_usgs_12.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_12
predictions = rf.predict(x_test(12)) 
r2_test = r2_score(y_test(12)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: -1.54


January 2022

In [17]:
pickled_model_usgs_1 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Interior_M-all-comids)/USGS_model_reduced_1_r2_0.8899_2022-09-19 13:06:02.482916-07:00.pkl', 'rb'))
print(pickled_model_usgs_1)

RandomForestRegressor(max_depth=5, max_features=16, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [18]:
rf_var = pickled_model_usgs_1.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_1
predictions = rf.predict(x_test(1)) 
r2_test = r2_score(y_test(1)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: -2.28


February 2022

In [19]:
pickled_model_usgs_2 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Interior_M-all-comids)/USGS_model_reduced_2_r2_0.8365_2022-09-19 13:07:00.530065-07:00.pkl', 'rb'))
print(pickled_model_usgs_2)

RandomForestRegressor(max_depth=5, max_features=16, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [20]:
rf_var = pickled_model_usgs_2.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_2
predictions = rf.predict(x_test(2)) 
r2_test = r2_score(y_test(2)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.54


March 2022

In [21]:
pickled_model_usgs_3 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Interior_M-all-comids)/USGS_model_reduced_3_r2_0.8178_2022-09-19 13:08:00.524088-07:00.pkl', 'rb'))
print(pickled_model_usgs_3)

RandomForestRegressor(max_depth=5, max_features=16, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [22]:
rf_var = pickled_model_usgs_3.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_3
predictions = rf.predict(x_test(3)) 
r2_test = r2_score(y_test(3)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.64


April 2022

In [23]:
pickled_model_usgs_4 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Interior_M-all-comids)/USGS_model_reduced_4_r2_0.7666_2022-09-19 13:08:58.985061-07:00.pkl', 'rb'))
print(pickled_model_usgs_4)

RandomForestRegressor(max_depth=5, max_features=16, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [24]:
rf_var = pickled_model_usgs_4.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_4
predictions = rf.predict(x_test(4)) 
r2_test = r2_score(y_test(4)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.95


May 2022